In [ ]:
!apt install pdfgrep libmagickwand-dev
!pip install pdftotree camelot-py pdf2image nltk

In [ ]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local

In [ ]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.6/site-packages/")

In [ ]:
!conda install -c conda-forge python=3.6 tesseract poppler

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Extracting Financial Data from the list of PDF's

In [ ]:
# import all the required libraries
import os
import pdftotree
import subprocess
from subprocess import check_output, Popen
from pdf2image import convert_from_path
import re
from PyPDF2 import PdfFileWriter, PdfFileReader
# import fitz
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
def get_num_pages(pdf_path):
    output = check_output(["pdfinfo", pdf_path]).decode()
    pages_line = [line for line in output.splitlines() if "Pages:" in line][0]
    num_pages = int(pages_line.split(":")[1])
    return num_pages

In [ ]:
# Cell for File Paths
pdf_dir = "/content/drive/My Drive/startup/interview_data/"
temp_dir = "/content/drive/My Drive/startup/test_jpg/"
text_dir = "/content/drive/My Drive/startup/all_text_cr/"
img_path = "/content/drive/My Drive/startup/imgpath.txt"
csv_file = "/content/drive/My Drive/startup/final.csv"
bse_companies = "/content/drive/My Drive/startup/bse_companies.csv"
large_pdfs = "/content/drive/My Drive/startup/large_pdfs/"

In [ ]:
# Since we are referring to the bse companies csv file, we need to store the map from different codes to company names
f = open(bse_companies,"rb")
bb_map = {}
nse_map = {}
bse_map = {}
# print(f.readlines())
for l in f.readlines():
  print(type(l),l)
  ld = l.decode('utf-8').split(',')
  if (ld[4] != ''):
    nse_map[ld[4]] = ld[2]
  if (ld[5] != ''):
    bb_map[ld[5]] = ld[2]
  if (ld[3] != ''):
    bse_map[ld[3]] = ld[2]
f.close()

In [ ]:
# Function to get text from pdf by OCR

def get_ocr_output(pdf_dir,temp_file):
  nfile = pdf_dir+temp_file
  pdf_id = nfile.strip('.pdf')

  filename = nfile
  num_pages = get_num_pages(filename)
  if num_pages > 150:
    process = Popen(['cp',str(filename), large_pdfs],
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    print(str(pdf_id)+" copied to large_pdfs folder")
  fp = open(img_path,"w")

  # Convert a PDF into a list of single page .jpg files
  # needed for Tesseract
  for num in range(1,num_pages+1):
    page = convert_from_path(filename, 500,first_page=num,last_page=num)
    page[0].save(temp_dir+temp_file+'_'+str(num)+'.jpg', 'JPEG')
    fp.write(temp_dir+temp_file+'_'+str(num)+'.jpg\n')
  fp.close()
  list_text_files = os.listdir(temp_dir)
  print("No of Pages in this pdf: "+str(len(list_text_files)))

  text_file = text_dir+str(temp_file.strip('.pdf'))
  
  # Apply tesseract OCR
  process = Popen(['tesseract', img_path, text_file, '--oem', '3', '-l','eng+hin', '-c', 'page_separator=♞'],
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  for i in list_text_files:
      os.remove(temp_dir+str(i))
  os.remove(img_path)
  # Need to return the path to the text file containg the text output
  return text_file+'.txt'

#### Get Persons

In [ ]:
# Function to get person names from a text file (single page)
def get_person_list(filename):
  f = open(test_txt,'r')
  text = 'While in France, Christine Lagarde along with Sahil Makhija and Bhrugesh Parsawala and RAVI SWAMINATHAN discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'
  text_test = f.read()
  i = 0
  persons = []

  # Extracting relevant person names in list format
  while i < len(classified_test):
    if 'PERSON' in classified_test[i][1]:
      j = i
      name = ""
      while 'PERSON' == classified_test[j][1]:
        name += classified_test[j][0]+" "
        j += 1
      i = j
      name = name[:-1]
      persons.append(name)
    i += 1
  if len(persons) > 0:
    print(persons)
    return ';'.join(persons)
  else:
    return "NA"

#### Get Rating

In [ ]:
# Page No has to start from 1
# convert a page in pdf to jpg
def get_pdfpage_jpg(filename,new_file_name,page_no):
  page = convert_from_path(filename, 500,first_page=page_no,last_page=page_no)
  page[0].save(new_file_name, 'JPEG')
  return

In [ ]:
# Function to get text data from image by OCR
def get_ocr_page(filename,new_txt_name,page_no):
  new_jpg_name = filename.strip('.pdf')+'.jpg'

  get_pdfpage_jpg(filename,new_txt_name+'.jpg',page_no)
  process = Popen(['tesseract', new_txt_name+'.jpg', new_txt_name, '--oem', '3', '-l','eng', '-c', 'page_separator=♞'],
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  # print(stdout)
  # print(stderr)
  return

In [ ]:
# Function to get the recommended rating from the pdf
def getRating(filename):

  # Check whether the pdf is unrated or not
  arg = ['grep','--max-count', '5', '-o', f"{'(NOT RATED|Under Review)'}", filename]
  companies_check = subprocess.run(arg, stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')[:-1]
  print(companies_check)
  if len(companies_check) > 0:
    return "NA"

  # List of rating keywords to be searched in the pdf
  keywords = ['BUY','Buy','SELL','Sell','HOLD','ADD','Add','REDUCE','Reduce','Hold','Neutral','NEUTRAL','Outperform','OUTPERFORM','Underperform','UNDERPERFORM','Suspended','SUSPENDED','UNDERPERFORMER','Underperformer','OUTPERFORMER','Outperformer','Accumulate','ACCUMULATE']
  keywords_regex = ""
  for i in range(len(keywords)):
    keywords_regex += keywords[i]
    if i <= len(keywords) - 1:
      keywords_regex += "|"
  keywords_regex = "\"[^\w]("+keywords_regex+")[^\w]\""

  f = open(filename,  "r")
  for l in f.readlines():
    for keyword in  keywords:
      if re.search(r"\W{}\W".format(keyword),l):
        res = keyword
        return res
  # In case the rating keyword is not found, leave blank
  return ""

#### Target Price

In [ ]:
# extract the word which denotes the price to be obtained
def get_money_amt(prices):
  for j in prices.split(' '):
    if any(char.isdigit() for char in j):
      return j

In [ ]:
# Out of all the amounts, get one which is maximum
# Applied across all target prices
def get_max_amt(amts):
  maxstr = ""
  maxa = -1
  print(amts)
  for amt in amts:
    a = ""
    if amt is not None:
      for char in amt:
        if char.isdigit():
          a = a + char
      if int(a) > maxa:
        maxstr = amt
        maxa = int(a)
  return maxstr

In [ ]:
# Save the first page of pdf as a separate pdf
def getFirstPage(file_path, new_file_path):
  inputpdf = PdfFileReader(open(file_path, "rb"))

  # if the pdf is encrypted
  # work with the original file itself
  if inputpdf.isEncrypted:
    return False
  output = PdfFileWriter()
  x = inputpdf.getPage(0)
  output.addPage(x)
  with open(new_file_path, "wb") as outputStream:
      output.write(outputStream)

In [ ]:
def fetch_target_price(cmd_tool,file_path):
  # For PDF data
  if cmd_tool == "pdfgrep":
    is_encrypted = getFirstPage(file_path, large_pdfs+"check_first_0.pdf")
    # if the pdf is encrypted
    # work with the original file itself
    if is_encrypted:
      nfile_path = file_path
    else:
      nfile_path = large_pdfs+"check_first_0.pdf"
    # if 
  # For Text Data
  else:
    nfile_path = file_path
  
  # Check whether the pdf is unrated or not
  arg = [cmd_tool,'--max-count', '5', '-o', f"{'(NOT RATED|Under Review)'}", nfile_path]
  companies_check = subprocess.run(arg, stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')[:-1]
  if len(companies_check) > 0:
    return "NA"

  # Price regex used for extracting the target price data
  price_regex = '([eE]xit [pP]rice|[tT]arget [pP]rice|[pP]rice [tT]arget|Fair Value|[pP]rice|[tT]arget[s]?|TP|TARGET|[pP]rice [oO]bjective|value the stock)[\t\ ]*(\([^/\n]*\))?:?[\t\ ]*-?[\t\ ]*(of|at|to)?[\t\ ]*(RS|Rs|INR|Inr|`|W|\|)?[.]?[\t\ ]*[0-9,.]+[\t\ ]*(RS|Rs|INR|Inr|`|W|\|)?'
  arg = [cmd_tool,'--max-count', '5', '-o', f"{price_regex}", file_path]
  companies = subprocess.run(arg, stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')[:-1]
  target_check = ['target','Target','TARGET','value','objective','Objective','value','TP','Fair','fair']
  target_achieved = False
  amts = []

  # If there are phrases containing target price, collect the price data
  # and compare with other target prices for the maximum one
  for l in range(len(companies)):
    # amts = []
    for t in target_check:
      if t in companies[l] and any(char.isdigit() for char in companies[l]):
        target_achieved = True
        amt = get_money_amt(companies[l])
        amts.append(amt)
        break
  if target_achieved:
    return ''.join(get_max_amt(amts).split(','))
  # If not the case, compare on others to get the maximum price data
  else:
    amts = []
    for l in range(len(companies)):
      amts.append(get_money_amt(companies[l]))
    res = get_max_amt(amts)
    return ''.join(res.split(','))

In [ ]:
# Wrapper function to handle both text and pdf data
def get_target_price(pdf_dir,temp_file,text_pdf_path):
  if text_pdf_path != "":
    # ToDo
    return fetch_target_price("grep",text_pdf_path)
    # pass
  else:
    nfile = pdf_dir+temp_file
    return fetch_target_price("pdfgrep",nfile)

#### Auditing Company

In [ ]:
# Function to get auditing company name
def get_aud_company(pdf_dir,temp_file,text_pdf_path):
  nfile = pdf_dir + temp_file
  # Candidate Regex to get auditing company from email ids of employees
  filtered = '[A-Za-z]+@[A-Za-z]+.[A-Za-z]+' # the list of regexes is shortened to one string, to keep the example simple.
  # Candidate Regex to get auditing company from its "pvt ltd" name
  filtered_ltd = '[A-Z][A-Za-z.]+[\t\ ]*([A-Za-z.]+[\t\ ]*)+([lL]td|[lL]imited)'
  arg = ['pdfgrep',  '-o',  f"{filtered}", nfile]
  arg_ltd = ['pdfgrep',  '-o',  f"{filtered_ltd}", nfile]
  process_match = subprocess.run(arg, stdout=subprocess.PIPE)
  process_match_ltd = subprocess.run(arg_ltd, stdout=subprocess.PIPE)
  emails = process_match.stdout.decode("utf-8").split('\n')
  emails_ltd = process_match_ltd.stdout.decode("utf-8").split('\n')

  # If candidate 1 matches
  if len(emails) > 1 and emails[0] != '':
    print(emails)
    return emails[0].split('@')[1].split('.')[0]
  # If candidate 2 matches
  elif len(emails_ltd) > 1 and emails_ltd[0] != '':
    while emails_ltd[-1] == '' and len(emails_ltd) > 1:
      emails_ltd.pop()
    if len(emails_ltd) == 0:
      return ''
    else:
      return emails_ltd[-1]

  elif text_pdf_path != "":
    # the pdf may not be recognizable by pdfgrep
    # in this case, we use tesseract ocr to extract company names
    # text_path = get_ocr_output(pdf_dir,temp_file)
    arg = ['grep',  '-o','-E',  f"{filtered}", text_pdf_path]
    process_match = subprocess.run(arg, stdout=subprocess.PIPE)
    emails_text = process_match.stdout.decode("utf-8").split('\n')
    print(emails_text)
    return emails_text[0].split('@')[1].split('.')[0]

#### Company being Audited

In [ ]:
# helper function to get intersection of two lists
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [ ]:
def get_company_name(pdf_dir,temp_file,text_pdf_path):
  if text_pdf_path != "":
    return "Negative"
    pass
  else:
    nfile = pdf_dir+temp_file
    # Getting company names from BloomBerg Codes
    bloomberg_regex = '[A-Z]+[\t\ ]*:?(IN|IB)[^A-Za-z0-9]'
    # Getting company names from NSE/BSE codes
    nse_regex = '(NSE|BSE)[\t\ ]*(SYMBOL|[Cc]ode|[tT]icker)?:?[\t\ ]*([A-Z]+|[0-9]+)[^a-z]'
    # If the above two don't work, then in some documents, the first occuring Alphabetic code in round caps and capital
    # would correspond to bloomberg codes
    bracket_bb_regex = '\([A-Z]+\)'
    arg = ['pdfgrep', '-o', f"{bloomberg_regex}", nfile]
    arg_nse = ['pdfgrep',  '-o',  f"{nse_regex}", nfile]
    arg_bb = ['pdfgrep',  '-o',  f"{bracket_bb_regex}", nfile]
    companies = subprocess.run(arg, stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')[:-1]
    # Preprocessing on bloomberg codes
    for i in range(len(companies)):
      if ':' in companies[i]:
        companies[i] = ' '.join(companies[i].split())
      if companies[i][-2:] == "IB":
        companies[i] = companies[i].split('IB')[0]+"IN"
      elif companies[i][-2:] != "IN":
        companies[i] = companies[i][:-1]
    companies_nse = subprocess.run(arg_nse, stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')[:-1]
    companies_bb = subprocess.run(arg_bb, stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')[:-1]
    # Case 1
    if len(companies) > 0 and intersection(companies,list(bb_map.keys())):# companies[0].split(' ')[0] not in ['CHANGE','IMPORTANT']:
      final_list = intersection(companies,list(bb_map.keys()))
      return ';'.join([bb_map[i] for i in final_list])
    # Case 2
    elif len(companies_nse) > 1 and not re.match('(NSE|BSE) INB',companies_nse[0]):
      if "NSEX" in companies_nse[0] and len(companies_nse) >= 2:
        companies_nse[0] = companies_nse[1]
      forbid = ["SYMBOL","Code","code",":","ticker",'Ticker']
      for w in forbid:
        if w in companies_nse[0]:
          companies_nse[0] = ' '.join(companies_nse[0].split(w))
      
      companies_nse[0] = ' '.join(companies_nse[0].split())
      # BSE codes contain only digits and NSE codes contain only capital letters
      # checking the company based on that.
      if not companies_nse[0][-1].isdigit() and not companies_nse[0][-1].isalpha():
        companies_nse[0] = companies_nse[0][:-1]
      print(companies_nse[0])
      code_arg = ' '.join(companies_nse[0][3:].split())
      if code_arg.isdigit():
        if code_arg in bse_map.keys():
          return bse_map[code_arg]
        else:
          return ""
      elif code_arg.isalpha():
        if code_arg in nse_map.keys():
          return nse_map[code_arg]
        else:
          return ""
    # Case 3
    else:
      if  (len(companies_bb) == 0):
        return ""
      else :
        code_arg = companies_bb[0][1:-1]+" IN"
        if code_arg in bb_map.keys(): 
          return bb_map[code_arg]
        else:
          return ""

#### Main Parsing

In [ ]:
list_files = sorted(os.listdir(pdf_dir))
flines = []
f = open(csv_file,'w')
for idx, nfile in enumerate(list_files):
  pdf_id = nfile.strip('.pdf')
  print(nfile)
  # Company report
  if "[CR]" in pdf_id:
    is_sector = "NO"
    rec_pdf = subprocess.run(['pdfgrep',  '-o',  f"the", pdf_dir+nfile], stdout=subprocess.PIPE).stdout.decode("utf-8").split('\n')
    if len(rec_pdf) == 1 and rec_pdf[0] == '':
      text_path = get_ocr_output(pdf_dir,nfile)
    else:
      text_path = ""
    get_ocr_page(pdf_dir+nfile,temp_dir+pdf_id+"_0",1)
    num_p = get_num_pages(pdf_dir+nfile)
    rating = getRating(temp_dir+pdf_id+"_0.txt")
    persons = get_person_list(temp_dir+pdf_id+"_0.txt")
    target_price = get_target_price(pdf_dir,nfile,text_path)
    aud_company = get_aud_company(pdf_dir,nfile)
    obj_company = get_company_name(pdf_dir,nfile,text_path)
    
  # Sector report
  elif "[Sector]" in pdf_id:
    is_sector = "YES"
    get_ocr_page(pdf_dir+nfile,temp_dir+pdf_id+"_0",1)
    aud_company = get_aud_company(pdf_dir,nfile)
    obj_company = "None"
    target_price = "NA"
    rating = "NA"
    persons = get_person_list(temp_dir+pdf_id+"_0.txt")
  # the remaining files are marked as others
  else:
    is_sector = "NO"
    aud_company = "Others"
    obj_company = "None"
    target_price = "NA"
    rating = "NA"
    persons = "NA"
  modified_nfile = ' '.join(pdf_id.split(','))+".pdf"
  fline = modified_nfile+","+aud_company+","+is_sector+","+obj_company+","+target_price+","+rating+","+persons'\n'
  print(fline)
  f.write(fline)
  flines.append(fline)
print(flines)
f.close()

AR032012.pdf
AR032012.pdf,Others,NO,None,NA,NA

ASHP_shortage_guide09.pdf
ASHP_shortage_guide09.pdf,Others,NO,None,NA,NA

AUTO-SALES-20140929-MOSL-SU-PG004.pdf
AUTO-SALES-20140929-MOSL-SU-PG004.pdf,Others,NO,None,NA,NA

Apna Supply Chain Weekly_Milind Shahane_05.02.15.pdf
Apna Supply Chain Weekly_Milind Shahane_05.02.15.pdf,Others,NO,None,NA,NA

Ascension-Industries-Modular-Design.pdf
Ascension-Industries-Modular-Design.pdf,Others,NO,None,NA,NA

Ashiana 2011-2012_Annual_Report.pdf
Ashiana 2011-2012_Annual_Report.pdf,Others,NO,None,NA,NA

Ashiana Housing - Concall -  Feb2010.pdf
Ashiana Housing - Concall -  Feb2010.pdf,Others,NO,None,NA,NA

Astral Poly Technik - EIf.pdf
Astral Poly Technik - EI.pdf,Others,NO,None,NA,NA

Audited Financial Results_30.06.2014.pdf
Audited Financial Results_30.06.2014.pdf,Others,NO,None,NA,NA

Avant Garde Wealth Mgmt - Quarterly letter - 1306.pdf
Avant Garde Wealth Mgmt - Quarterly letter - 1306.pdf,Others,NO,None,NA,NA

Avant Garde Wealth Mgmt - Quarterly l

['cirumalla@bksec.com', '']
[] [] []
BNK_051616_25242 [CR].pdf,bksec,NO,,865,BUY

BNK_090215_195059 [CR].pdf
[]
Found: BUY
[]


['Price                                      Rs 683   ', 'Target Price                               Rs 865   ', 'target price of Rs 865 ', 'target price of Rs 865.', 'price, ']
['865', '865', '865.']
['cirumalla@bksec.com', '']
['GFLC IN'] [] ['(GFL)', '(MT)', '(PFOA)', '(GWP)', '(GFL)', '(CDM)', '(GFL)', '(PCPIR)', '(COP)', '(ODP)', '(GWP)', '(PFOA)', '(HF)', '(PVDF)', '(FEP)', '(F)', '(HF)', '(PTFE)', '(PTFE)', '(UHMWPE)', '(IWL)', '(WTG)', '(ECS)', '(AD)', '(L)', '(L)', '(L)', '(L)']
BNK_090215_195059 [CR].pdf,bksec,NO,Gujarat Fluorochemicals Ltd.,865,BUY

BNK_SONA@IN_021715_161235 [CR].pdf
[]
Not Found?
[]


['Price: Rs 53                           ', 'Target price (Rs)                       62  ', 'price of Rs 52, ', 'target price of Rs 62 ']
['62', '62']
['jayaraj@bksec.com', 'rajpara@bksec.com', '']
['SONA IN', ''] [] ['(SONA)', '(EPS)', '(EPAM)', '(L)']
BNK_SONA@IN_021715_161235 [CR].pdf,bksec,NO,,62,

BOAML-BhagMilkhaBhag.pdf
BOAML-BhagMilkhaBhag.pdf,Others,NO,None,NA,NA

BOAML-StagnatingIndicators.pdf
BOAML-StagnatingIndicators.pdf,Others,NO,None,NA,NA

BP Equities- National Buildings Construction Corporation Ltd NBCC- Short note_11th April, 2014 [CR].pdf
[]
Found: BUY
[]
['Target Price (Rs)               232', 'target price of Rs. 232. ']
['232', '232.']


['NBCC IN', ''] ['BSE code                    534309', ''] ['(NBCC)', '(PMC)']
BP Equities- National Buildings Construction Corporation Ltd NBCC- Short note_11th April  2014 [CR].pdf,BP Wealth MANAGEMENT Pvt. Ltd,NO,NBCC (India) Ltd.,232,BUY

BP Wealth- Kitex Garments Limited [CR].pdf
[]
Found: BUY
[]


...

['Price (`): 99', 'Target price (`): 110', 'TP at `110 ', 'TP (`110 from `120 earlier). W', 'target price, ']
['110', '`110', '(`110']
['mahesh@kotak.com', 'chawathe@kotak.com', 'ghosh@kotak.com', 'singh@kotak.com', 'research@kotak.com', 'goenka@kotak.com', 'compliance@kotak.com', 'securities@kotak.com', 'demat@kotak.com', 'escalation@kotak.com', 'servicehead@kotak.com', 'compliance@kotak.com', 'ks@kotak.com', '']
['CIN'] ['NSE INB/', 'BSE INB '] ['(KVB)', '(X)', '(X)', '(LHS)', '(RHS)', '(LHS)', '(RHS)', '(UK)', '(E)', '(UK)', '(UK)', '(UK)', '(BSE)', '(NSE)', '(NSDL)', '(CDSL)', '(AMFI)', '(E)']
[CR] [Kotak] Karur Vysya Bank  July 25  2018.pdf,kotak,NO,Karur Vysya Bank Ltd.,110,ADD

[CR] [Kotak] Marico, May 3, 2018.pdf
[]
Found: ADD
[]


['Price (`): 317', 'Target price (`): 345', 'target of `345', 'target price, ', 'price target, ']
['345', '`345']
['chordia@kotak.com', 'doshi@kotak.com', 'sethi@kotak.com', 'research@kotak.com', 'goenka@kotak.com', 'compliance@kotak.com', 'securities@kotak.com', 'demat@kotak.com', 'escalation@kotak.com', 'servicehead@kotak.com', 'compliance@kotak.com', 'ks@kotak.com', '']
['CIN'] ['NSE INB/', 'BSE INB '] ['(MRCO)', '(X)', '(X)', '(UK)', '(E)', '(UK)', '(UK)', '(UK)', '(BSE)', '(NSE)', '(NSDL)', '(CDSL)', '(AMFI)', '(E)']
[CR] [Kotak] Marico  May 3  2018.pdf,kotak,NO,Marico Ltd.,345,ADD

[CR] [Kotak] Nestle India, October 26, 2018.pdf
[]
Found: ADD
[]
['Price (`): 9,540', 'Target price (`): 10,600', 'target to `10,600 ', 'target of `10,600', 'target price, ']
['10,600', '`10,600', '`10,600']


['chordia@kotak.com', 'doshi@kotak.com', 'sethi@kotak.com', 'research@kotak.com', 'goenka@kotak.com', 'compliance@kotak.com', 'securities@kotak.com', 'demat@kotak.com', 'escalation@kotak.com', 'servicehead@kotak.com', 'compliance@kotak.com', 'ks@kotak.com', '']
['CIN'] [] ['(NEST)', '(X)', '(X)', '(UK)', '(E)', '(UK)', '(UK)', '(UK)', '(BSE)', '(NSE)', '(NSDL)', '(CDSL)', '(AMFI)', '(E)']
[CR] [Kotak] Nestle India  October 26  2018.pdf,kotak,NO,Nestle India Ltd.,10600,ADD

[CR] ar_jyothy.pdf
[]
Found: Buy
[]
['Target Price: `210', 'Price: `162', 'target price to `210 ', 'value the stock                                                    120', 'price target of `210 ']
['`210', '`210', '120', '`210']
['shirishpardeshi@rathi.com', '']
['JYL IN'] ['BSE FMCG ', 'BSE FMCG', ''] ['(HUL)', '(LAB)', '(MAT)', '(ARSSBL)', '(HK)', '(HK)', '(HK)']
[CR] ar_jyothy.pdf,rathi,NO,Jyothy Laboratories Ltd.,`210,Buy

[Oth] 07102014115602MMDR Act 1957_10052012 (2).pdf
[Oth] 07102014115602MMDR Act 1957_10052

['pujara@enam.com', 'madon@enam.com', '']
['VOLT IB'] [] ['(MEP)']
enam_voltas [CR].pdf,enam,NO,Mid East Portfolio Management Ltd.,74,SELL

eq_HAVELLS_upd9_10Jun2013.pdf
eq_HAVELLS_upd9_10Jun2013.pdf,Others,NO,None,NA,NA

expertprint.pdf
expertprint.pdf,Others,NO,None,NA,NA

federal.pdf
ederal.pdf,Others,NO,None,NA,NA

find-information.pdf
ind-information.pdf,Others,NO,None,NA,NA

funding-strategies.pdf
unding-strategies.pdf,Others,NO,None,NA,NA

g32n06a.pdf
g32n06a.pdf,Others,NO,None,NA,NA

http___www.edelresearch.com_rpt_showPdf.aspx_id=20806&reportname=_BRAVEHEART_SERIES-Zee_Entertainment_Enterprises.pdf
http___www.edelresearch.com_rpt_showPdf.aspx_id=20806&reportname=_BRAVEHEART_SERIES-Zee_Entertainment_Enterprises.pdf,Others,NO,None,NA,NA

iEnergy+Theni+Wind+farm+RAC.pdf
iEnergy+Theni+Wind+farm+RAC.pdf,Others,NO,None,NA,NA

iSec- VRL Logistics [CR].pdf
[]
Found: ADD
[]
['Target price Rs332                                                           ']
['Rs332']
['mathew@icicisecurit

['Target Price: Rs902                                                                                               ', 'target price of Rs902. ', 'target price of Rs902 ', 'target price, ']
['Rs902', 'Rs902.', 'Rs902']
['kedia@centrum.co', 'grievances@centrum.co', 'compliance@centrum.co', '']
['JUST IN'] ['NSE CNX ', 'BSE: INB0', 'NSE: INB2', 'NSE: INF2'] ['(IDC)', '(RHS)', '(RHS)', '(RHS)', '(DND)', '(SMS)', '(SAIF)', '(BSE)', '(NSE)', '(CDSL)', '(E)']
just dial - 240913 - centrum [CR].pdf,centrum,NO,Just Dial Ltd.,Rs902,BUY

kotak ecommerce.pdf
kotak ecommerce.pdf,Others,NO,None,NA,NA

lbs061812appendixG.pdf
lbs061812appendixG.pdf,Others,NO,None,NA,NA

macroperspective.pdf
macroperspective.pdf,Others,NO,None,NA,NA

make-consolidated-january19-2013-to-march31-2014.pdf
make-consolidated-january19-2013-to-march31-2014.pdf,Others,NO,None,NA,NA

mayur2 [CR].pdf
[]
Not Found?
[]
['Target Price        469.00', 'price of Rs.438.00, ']
['469.00']
['info@firstcallindia.com', 'info@firstcallind